# Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/data/*.zip .
!unzip /content/sudoku.zip
!unzip /content/sudoku_test.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  /content/sudoku.zip
replace sudoku.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  /content/sudoku_test.zip
replace sudoku.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
!git clone https://github.com/cloughurd/drl-sudoku.git
!mv drl-sudoku/data/* .

fatal: destination path 'drl-sudoku' already exists and is not an empty directory.
mv: cannot stat 'drl-sudoku/data/*': No such file or directory


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import librosa
import os
import gc
import wave
import struct
import math
import contextlib
from mido import MidiFile
from mido.messages.messages import Message
import mido
from typing import List
from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler


from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

# Create dataloader

In [0]:
def normalize_mono_grid(m):
  return (m / 9.0) - 0.5

In [0]:
from dataloader import *

# Define Model

In [0]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(self.shape)

In [0]:
class ToSudokuRange(nn.Module):
  def __init__(self):
    super(ToSudokuRange, self).__init__()
    self.sigmoid = nn.Sigmoid()
    self.net = lambda x: (9 * self.sigmoid(x)) + 0.5
  
  def forward(self, x):
    return self.net(x)

In [0]:
class SolverLayer(nn.Module):
  def __init__(self, in_filters:int, hidden_filters:int, out_filters:int):
    super(SolverLayer, self).__init__()

    self.initial_normalization = nn.InstanceNorm2d(in_filters) # Normalize every individual game within its filters. Not positive this is a good idea... maybe use the 3d version?

    self.HorizontalDependencies = nn.Sequential(        
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(9, 1)), # Output is 1, 9.
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81), 
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.VerticalDependencies = nn.Sequential(
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(1, 9)), # Output is 9, 1.
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81), 
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.QuadrantDependencies = nn.Sequential(
        nn.Conv2d(in_filters, hidden_filters, kernel_size=(3, 3), stride=3),
        nn.LeakyReLU(),
        Reshape((-1, hidden_filters, 9)),
        # Make into a full board shape that can be recombined... Maybe?
        nn.Linear(9, 81), 
        Reshape((-1, hidden_filters, 9, 9)),
        nn.LeakyReLU()
    )

    self.Reduce = nn.Sequential(
        nn.Conv2d(hidden_filters * 3, out_filters, kernel_size=(1, 1)), # Look at each cell only, without neighbors; neighbors have already been considered.
        nn.LeakyReLU()
    )

  def forward(self, x):
    x = self.initial_normalization(x)
    horizontal_result = self.HorizontalDependencies(x)
    vertical_result = self.VerticalDependencies(x)
    quadrant_result = self.QuadrantDependencies(x)

    combined = torch.cat((horizontal_result, vertical_result, quadrant_result), dim=1)
    reduced = self.Reduce(combined)
    return reduced
    


In [0]:
class MonoModel(nn.Module):
  def __init__(self, solver_depth:int):
    super(MonoModel, self).__init__()
    self.net = nn.Sequential(
        # Give a channel dimension
        Reshape((-1, 1, 9, 9)),
        SolverLayer(1, 9, 9),
        *[ SolverLayer(9, 9, 9) for _ in range(solver_depth - 2)],
        SolverLayer(9, 9, 1),
        Reshape((-1, 9, 9)),
        ToSudokuRange()
    )

  def forward(self, x):
    return self.net(x)

In [0]:
def puzzle_exactifier(p):
  return torch.round(p)

def puzzle_masker(attempts, starting_puzzles):
  # print(attempts.size(), starting_puzzles.size())
  assert attempts.size() == starting_puzzles.size()
  num_filled_in_solution = torch.sum(starting_puzzles != 0).item()
  attempts = torch.where(starting_puzzles == 0, attempts, starting_puzzles)
  return (attempts, starting_puzzles, num_filled_in_solution)

def solved_accuracy(attempts, solutions, starting_puzzles):
  assert attempts.size() == solutions.size()
  masked_puzzle, _, _ = puzzle_masker(attempts, starting_puzzles)
  # print(masked_puzzle, solutions, starting_puzzles)
  num_puzzles = attempts.size(0)
  num_correct = (masked_puzzle.eq(solutions).sum(1).sum(1) == 9).sum().item()
  return num_correct / num_puzzles

def cell_accuracy(attempts, solutions, starting_puzzles):
  assert attempts.size() == solutions.size()
  masked_puzzle, _, num_filled = puzzle_masker(attempts, starting_puzzles)
  # print("\nAttempts:\n", attempts, "\nMasked\n", masked_puzzle, "\nInitial\n", starting_puzzles, "\nSolution\n", solutions)
  num_cells = attempts.numel()
  num_correct = masked_puzzle.eq(solutions).sum().item()
  num_guessed_correctly = num_correct - num_filled
  num_to_guess = num_cells - num_filled
  # print(f"--- {num_cells} - {num_correct} - {num_cells} - {num_filled} ---")
  return num_guessed_correctly / num_to_guess


# Training Loop

In [0]:
def train(model:MonoModel, criterion:torch.nn.modules.loss._Loss, optimizer:torch.optim.Optimizer, train_loader:DataLoader, valid_loader:DataLoader, num_epochs:int, valid_frequency:int=5):
  loop = tqdm(total=num_epochs * len(train_loader) + (num_epochs // valid_frequency) * len(valid_loader), position=0)

  # The loss of the last training and validation iteration, respectively.  
  train_loss = None
  valid_loss = None

  # The puzzle accuracy of the last training and validation itersions. ( # puzzles right / # puzzles total )
  train_accuracy = None
  valid_accuracy = None

  # The cell accuracy of the last training and validation iterations ( # filled cells right / # num fillable cells )
  train_inner_acc = None
  valid_inner_acc = None

  for e in range(num_epochs):

    training_losses = []
    training_accuracies = []
    training_cell_accuracies = []

    loop.set_description(f"[Training] Epoch: {e}. Loss: {valid_loss}/{train_loss}. Total Accuracy: {train_accuracy}/{valid_accuracy}. Inner Accuracy: {train_inner_acc}/{valid_inner_acc}")

    for puzzle, solution in train_loader:
      puzzle, solution = puzzle.float().squeeze(1), solution.float() + 1
      initial_puzzle = puzzle.clone()

      optimizer.zero_grad()
      attempt = model(puzzle)
      loss = criterion(attempt, solution)

      loss.backward()
      optimizer.step()

      train_loss = loss.item()
      training_losses.append(train_loss)

      # Compute accuracies
      exactified = puzzle_exactifier(attempt)

      solve_accuracy = solved_accuracy(exactified, solution, initial_puzzle)
      c_acc = cell_accuracy(exactified, solution, initial_puzzle)

      train_accuracy = solve_accuracy
      training_accuracies.append(train_accuracy)

      train_inner_acc = c_acc
      training_cell_accuracies.append(train_inner_acc)

      loop.set_description(f"[Training] Epoch: {e}. Loss: {valid_loss}/{train_loss}. Total Accuracy: {valid_accuracy}/{train_accuracy}. Inner Accuracy: {train_inner_acc}/{valid_inner_acc}")
      loop.update()




In [0]:
model = MonoModel(solver_depth=11)
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()
train_loader = get_loader(root="/content/", batch_size=2, mono=True, train=True)
valid_loader = []
num_epochs = 5

In [37]:
train(model, criterion, optimizer, train_loader, valid_loader, num_epochs, 1)

[Training] Epoch: 0. Loss: None/6.669667720794678. Total Accuracy: None/0.0. Inner Accuracy: 0.1282051282051282/None:   0%|          | 529/22500000 [00:32<372:07:29, 16.80it/s]

KeyboardInterrupt: ignored

# TEST ACCURACY METRIC METHODS

In [101]:
j = torch.Tensor([[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])
k = torch.Tensor([[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])
l = torch.Tensor([[[2,2,2],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]],[[1,1,1],[2,2,2],[3,3,3]]])

z = torch.zeros((3,3,3))
f = z.clone()
f[:,:,0] = k[:,:,0]
g = z.clone()
g[:,:,0] = l[:,:,0]

tensor([[[2., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]]])


In [104]:
print(l)
print(cell_accuracy(j, l, g))

tensor([[[2., 2., 2.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]]])
tensor([[[2., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]]]) tensor([[[2., 2., 2.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]],

        [[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]]]) tensor([[[2., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]],

        [[1., 0., 0.],
         [2., 0., 0.],
         [3., 0., 0.]]])
Correct: <class 'int'>
Num cells: <class 'int'>
0.5925925925925926


In [68]:
f

tensor([[[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]],

        [[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]],

        [[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]])

In [123]:
x.size()

torch.Size([1, 1, 9, 9])

In [28]:
for i, (p, s) in enumerate(train_loader):
  print(f"{i}: \n{p}\n{s}")
  if i > 4:
    break

0: 
tensor([[[[0., 2., 0., 5., 3., 0., 0., 1., 7.],
          [0., 4., 6., 0., 0., 0., 3., 0., 5.],
          [0., 5., 0., 1., 8., 6., 2., 4., 0.],
          [2., 3., 4., 8., 0., 7., 9., 5., 0.],
          [5., 0., 0., 0., 0., 0., 7., 2., 8.],
          [7., 9., 8., 0., 0., 0., 4., 6., 3.],
          [0., 0., 0., 0., 5., 0., 1., 0., 0.],
          [0., 8., 3., 0., 0., 0., 5., 0., 0.],
          [0., 0., 0., 7., 0., 0., 0., 3., 4.]]],


        [[[9., 6., 0., 0., 0., 0., 2., 3., 0.],
          [2., 8., 7., 4., 1., 3., 9., 6., 0.],
          [3., 0., 1., 0., 9., 2., 8., 0., 7.],
          [6., 0., 9., 0., 7., 1., 3., 8., 0.],
          [4., 0., 5., 0., 2., 8., 7., 9., 6.],
          [0., 7., 3., 9., 0., 0., 0., 0., 2.],
          [0., 0., 0., 8., 0., 6., 1., 0., 9.],
          [0., 0., 8., 0., 5., 0., 6., 0., 0.],
          [0., 9., 0., 1., 0., 7., 0., 2., 0.]]]], dtype=torch.float64)
tensor([[[7., 1., 8., 4., 2., 3., 5., 0., 6.],
         [0., 3., 5., 8., 6., 1., 2., 7., 4.],
         [

In [0]:
x, y = next(iter(train_loader))

In [33]:
print(x[0],"\n", y[0])

tensor([[[2., 0., 0., 4., 7., 3., 0., 8., 0.],
         [0., 0., 0., 0., 9., 0., 0., 6., 4.],
         [0., 5., 3., 0., 0., 0., 9., 0., 0.],
         [0., 8., 5., 0., 4., 0., 7., 3., 0.],
         [0., 0., 4., 0., 6., 8., 0., 1., 0.],
         [0., 0., 2., 0., 5., 0., 0., 0., 8.],
         [0., 0., 0., 6., 0., 7., 0., 0., 0.],
         [0., 0., 8., 9., 0., 0., 0., 4., 0.],
         [0., 2., 7., 8., 0., 0., 0., 0., 9.]]], dtype=torch.float64) 
 tensor([[1., 8., 5., 3., 6., 2., 4., 7., 0.],
        [7., 6., 0., 4., 8., 1., 2., 5., 3.],
        [3., 4., 2., 0., 7., 5., 8., 6., 1.],
        [0., 7., 4., 1., 3., 8., 6., 2., 5.],
        [8., 2., 3., 6., 5., 7., 1., 0., 4.],
        [6., 5., 1., 2., 4., 0., 3., 8., 7.],
        [4., 3., 8., 5., 0., 6., 7., 1., 2.],
        [2., 0., 7., 8., 1., 4., 5., 3., 6.],
        [5., 1., 6., 7., 2., 3., 0., 4., 8.]], dtype=torch.float64)


In [31]:
y

tensor([[[1., 8., 5., 3., 6., 2., 4., 7., 0.],
         [7., 6., 0., 4., 8., 1., 2., 5., 3.],
         [3., 4., 2., 0., 7., 5., 8., 6., 1.],
         [0., 7., 4., 1., 3., 8., 6., 2., 5.],
         [8., 2., 3., 6., 5., 7., 1., 0., 4.],
         [6., 5., 1., 2., 4., 0., 3., 8., 7.],
         [4., 3., 8., 5., 0., 6., 7., 1., 2.],
         [2., 0., 7., 8., 1., 4., 5., 3., 6.],
         [5., 1., 6., 7., 2., 3., 0., 4., 8.]],

        [[3., 5., 2., 1., 8., 4., 7., 6., 0.],
         [6., 1., 4., 7., 3., 0., 8., 5., 2.],
         [0., 8., 7., 2., 6., 5., 1., 4., 3.],
         [8., 0., 6., 4., 7., 1., 2., 3., 5.],
         [4., 7., 1., 5., 2., 3., 6., 0., 8.],
         [2., 3., 5., 6., 0., 8., 4., 7., 1.],
         [7., 2., 8., 0., 5., 6., 3., 1., 4.],
         [1., 6., 0., 3., 4., 2., 5., 8., 7.],
         [5., 4., 3., 8., 1., 7., 0., 2., 6.]]], dtype=torch.float64)